In [ ]:
import databaseOps, fields, matching, analyze
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import pandas as pd
import json, sqlite3, pprint, re
from bs4 import BeautifulSoup, Tag

In [ ]:
conn = databaseOps.connectToDb()
cur = conn.cursor()

In [ ]:
df = pd.read_sql_query("select * from form4Head;", conn)
fours = df[df['documentType']=='4']['accNum'].values
fourAs = df[df['documentType']=='4/A']['accNum'].values
uniqueFours = np.unique(fours)
uniqueFourAs = np.unique(fourAs)

In [4]:
### Find changes for 4/a idx
# matches = matching.get4ATo4Matches(uniqueFourAs)
# changeDic = matching.get4ATo4Changes(uniqueFourAs)
with open('2016Q4_2017Q4_3MCOMatches.json') as data_file:
    matches = json.load(data_file)
with open('2016Q4_2017Q4_3MCOChanges.json') as data_file:
    changeDic = json.load(data_file)

In [5]:
# debug: see analyzeDictionary
# analysis = analyze.runAnalysis(changeDic)
# numAnalyses, numFieldNames, numRowIds = analyze.getAllAnalyses(changeDic, "numerical", toVect=True)
# print(json.dumps(analysis, indent=2))

In [6]:
numLabels, catLabels = analyze.getLabels(changeDic)

/Users/Star/Desktop/Academics/Haas Web Dev/SEC/analyze.py:124: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 124 of the file /Users/Star/Desktop/Academics/Haas Web Dev/SEC/analyze.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  beforeSoup = BeautifulSoup(beforeVal)
/Users/Star/Desktop/Academics/Haas Web Dev/SEC/analyze.py:131: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warn

In [7]:
numLabels["ndt"]

{'featureNames': ['amountChanged',
  'changeDirection',
  'changeType',
  'dataType',
  'fieldName',
  'percentChange'],
 'fieldName': array([2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2,
        0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2,
        0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0,
        2, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0,
        2, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2,
        0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2,
        0, 2, 0, 2, 0, 2, 0]),
 'labels': array([[ 8.12800000e+03,  1.00000000e+00,  5.10000000e+01,
          0.00000000e+00,  2.00000000e+00,  4.73470000e+01],
        [ 0.00000000e+00,  0.00000000e+00,  3.00000000e+01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 8.35900000e+03,  1.00000000e+00,  4.10000000e+01,
          0.00000000e+00,  2.00000000e+00,  4.86920000e+01],
        [ 0.000000

In [8]:
defaultVal = {
  'amountChanged': 0,
  'changeDirection': -1,
  'changeType': -1,
  'dataType': -1,
  'fieldName': -1,
  'percentChange': 0    
}
defaultNumVal = {
    'transactionShares': 0,
   'transactionPricePerShare': 0,
   'sharesOwnedFollowingTransaction': 0, 
}

In [11]:
# Append labels to table

table = pd.read_sql_query("select * from form4NDT;", conn)

# init empty columns for labels
for feature in numLabels["ndt"]['featureNames']:
    if feature in fields.numericalAnalysisType:
        table[feature] = np.zeros(len(table)) 
    else:
        table[feature] = np.nan

# Rid extra tags, map typedCategorical to ints
def cleanData(row):
    for i in range(len(row)):
        if type(row[i]) == str:
            row[i] = re.sub("<.*?>", "", row[i])
    for dateType in fields.date:
        if dateType in row and type(row[dateType]) == str:
            row[dateType] = row[dateType].replace("-", "")

    for col in table.columns:
        if col in fields.typedCategoricalMap:
            val = row[col]
            if val in fields.typedCategoricalMap[col]:
                row[col] = int(fields.typedCategoricalMap[col].index(val))
    return row

# Add label in table
idxMap = dict()
def fillInWithConstant(row): 
    idx = np.where(numLabels["ndt"]["rowIds"]["4"] == row['nDTId'])[0]
    if len(idx) > 0:
        if row['nDTId'] in idxMap:
            rowIdx = idxMap[row['nDTId']]
        else:
            rowIdx = 0
        idxMap[row['nDTId']] = rowIdx + 1
        for i in range(len(numLabels["ndt"]['featureNames'])):
            labels = numLabels["ndt"]["labels"][:,i]    
            label = labels[idx[rowIdx]]
            row[numLabels["ndt"]['featureNames'][i]] = label
    else:
        for i in range(len(numLabels["ndt"]['featureNames'])):
            feature = numLabels["ndt"]['featureNames'][i]
            row[feature] = defaultVal[feature]
            
    return row

table = table.apply(cleanData, axis=1) 
table = table.fillna(defaultNumVal)
table = table.apply(fillInWithConstant, axis=1) 
table = table.fillna(-1) 
table.head()

,accNum,nDTId,rowNumber,securityTitle,transactionDate,transactionFormType,transactionCode,equitySwapInvolved,transactionTimelines,transactionShares,...,natureOfOwnership,type,documentType,footNoteId,amountChanged,changeDirection,changeType,dataType,fieldName,percentChange
0,66740/0001127602-16-065866,66740/0001127602-16-065866-1,1,Common Stock,20161101,4,7.0,0.0,0.0,11205,...,-1,transaction,4,-1,0.0,-1.0,-1.0,-1.0,-1.0,0.0
1,66740/0001127602-16-065866,66740/0001127602-16-065866-2,2,Common Stock,20161101,4,5.0,0.0,0.0,5401,...,-1,transaction,4,-1,0.0,-1.0,-1.0,-1.0,-1.0,0.0
2,66740/0001127602-16-066334,66740/0001127602-16-066334-1,1,Common Stock,20161104,4,1.0,0.0,0.0,200,...,-1,transaction,4,-1,0.0,-1.0,-1.0,-1.0,-1.0,0.0
3,66740/0001127602-16-066334,66740/0001127602-16-066334-2,2,Common Stock,20161104,4,1.0,0.0,0.0,200,...,-1,transaction,4,-1,0.0,-1.0,-1.0,-1.0,-1.0,0.0
4,66740/0001127602-16-066334,66740/0001127602-16-066334-3,3,Common Stock,20161104,4,1.0,0.0,0.0,300,...,-1,transaction,4,-1,0.0,-1.0,-1.0,-1.0,-1.0,0.0


In [12]:
# create data table for training
data_col = [col for col in list(table.columns) if col not in numLabels["ndt"]['featureNames'] and col not in matching.ndtDropField]
data_table = table[data_col].copy()
categorical_col = [col for col in data_col if col in fields.categorical]
numerical_col = [col for col in data_col if col not in fields.categorical]

records = data_table[categorical_col].to_dict(orient='records')
vect = DictVectorizer(sparse=False)
featureVects = vect.fit_transform(records)
colnames = vect.get_feature_names() #get name of cols

features = pd.DataFrame(data_table[numerical_col])
for i in range(len(colnames)):
    features[colnames[i]] = featureVects[:, i]
data_val = features.values

In [17]:
# Create labels for training

### set analysis of changeDictionary as labels
### see if can predict ['amountChanged', 'changeDirection', 'changeType', 'dataType', 'fieldName', 'percentChange']
label_table =  table[numLabels["ndt"]['featureNames']].copy()

### see if can predict 'directOrIndirectOwnership' 
# label_table =  table[['directOrIndirectOwnership']].copy()

label_val = label_table.values.astype(int)

In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes"]#, "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB()]#,QuadraticDiscriminantAnalysis()]

In [19]:
# need to use kfold validation bc sets are so small
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(data_val)

for label_idx in range(label_val.shape[1]):
    for train_idx, valid_idx in kf.split(data_val):
        train_data, valid_data = data_val[train_idx], data_val[valid_idx]
        train_label, valid_label = label_val[:,label_idx][train_idx], label_val[:,label_idx][valid_idx]

        for name, clf in zip(names, classifiers):
            clf.fit(train_data, train_label)
            train_score = clf.score(train_data, train_label)
            valid_score = clf.score(valid_data, valid_label)
            print(name, label_table.columns[label_idx], train_score, valid_score)
            break
        
    print('\n')

Nearest Neighbors amountChanged 0.9381443298969072 0.9836065573770492
Nearest Neighbors amountChanged 0.9505154639175257 0.9262295081967213
Nearest Neighbors amountChanged 0.9423868312757202 0.9586776859504132
Nearest Neighbors amountChanged 0.9423868312757202 0.9669421487603306
Nearest Neighbors amountChanged 0.9629629629629629 0.8842975206611571


Nearest Neighbors changeDirection 0.9463917525773196 0.9426229508196722
Nearest Neighbors changeDirection 0.9567010309278351 0.9016393442622951
Nearest Neighbors changeDirection 0.9526748971193416 0.8925619834710744
Nearest Neighbors changeDirection 0.9465020576131687 0.9338842975206612
Nearest Neighbors changeDirection 0.9444444444444444 0.9421487603305785


Nearest Neighbors changeType 0.9463917525773196 0.9344262295081968
Nearest Neighbors changeType 0.9484536082474226 0.9016393442622951
Nearest Neighbors changeType 0.9465020576131687 0.9173553719008265
Nearest Neighbors changeType 0.9382716049382716 0.9338842975206612
Nearest Neighbors 

In [ ]:
serverChanges = {
  "1432178/0001225208-17-004702": {
    "ndt": {
      "1-transaction": {
        "transactionDate": {
          "4": {
            "value": "<value>2017-02-24</value>"
          },
          "4A": {
            "value": "<value>2017-02-23</value>"
          }
        }
      }
    },
    "accNum": "885725/0001225208-17-004574",
    "head": {},
    "dt": {}
  },
  "1430523/0001493152-17-000832": {
    "ndt": {
      "1-transaction": {
        "transactionPricePerShare": {
          "4": {
            "value": "<value>0.006</value><footnoteId id=\"F4\"/>",
            "F4": "On January 10, 2017, the reporting person exercised a warrant to purchase 11,581,467 shares of common stock for $0.006 per share. The reporting person paid the exercise price on a cashless basis, resulting in a withholding of 2,331,838 of the warrant shares to pay the exercise price and issuing to the reporting person the remaining 9,249,629 shares."
          },
          "4A": {
            "F2": "On January 10, 2017, the Reporting Person exercised a warrant to purchase 11,581,467 shares of the Issuer's common stock for $0.006 per share. The Reporting Person paid the exercise price on a cashless basis resulting in a withholding of 2,331,838 of the warrant shares to pay the exercise price resulting in the issuance of 9,249,629 shares.",
            "value": "<value>0.006</value><footnoteId id=\"F2\"/>"
          }
        },
        "transactionShares": {
          "4": {
            "value": "<value>11581467</value>"
          },
          "4A": {
            "value": "<value>11581467</value><footnoteId id=\"F1\"/>",
            "F1": "As of January 10, 2017, the Issuer is unable to issue the shares to the Reporting Person because the Issuer does not have sufficient authorized capital stock for the issuance, but the issuance of the shares has been irrevocably authorized. The Issuer will issue the shares upon the effective date of the Issuer's one (1) for two hundred (200) reverse stock split."
          }
        }
      },
      "2-transaction": {
        "transactionShares": {
          "4": {
            "value": "<value>2331838</value>"
          },
          "4A": {
            "value": "<value>2331828</value>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>9249629</value>"
          },
          "4A": {
            "value": "<value>9249629</value><footnoteId id=\"F1\"/>",
            "F1": "As of January 10, 2017, the Issuer is unable to issue the shares to the Reporting Person because the Issuer does not have sufficient authorized capital stock for the issuance, but the issuance of the shares has been irrevocably authorized. The Issuer will issue the shares upon the effective date of the Issuer's one (1) for two hundred (200) reverse stock split."
          }
        }
      }
    },
    "accNum": "1693901/0001493152-17-000427",
    "head": {
      "1": {
        "rptOwnerStreet2": {
          "4": {
            "value": ""
          },
          "4A": {
            "value": "9711 WASHINGTONIAN BLVD., #550"
          }
        },
        "rptOwnerStreet1": {
          "4": {
            "value": "9711 WASHINGTON BLVD., #550"
          },
          "4A": {
            "value": "C/O REALBIZ MEDIA GROUP, INC."
          }
        },
        "periodOfReport": {
          "4": {
            "value": "2017-01-02"
          },
          "4A": {
            "value": "2017-01-10"
          }
        }
      }
    },
    "dt": {}
  },
  "1238874/0001209191-17-002149": {
    "ndt": {
      "2-transaction": {
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>1690</value>"
          },
          "4A": {
            "value": "<value>1690</value><footnoteId id=\"F1\"/>",
            "F1": "Amount includes 1,110 of unvested restricted stock units"
          }
        }
      }
    },
    "accNum": "1013488/0001209191-17-001912",
    "head": {},
    "dt": {}
  },
  "1213483/0001225208-17-002187": {
    "ndt": {
      "1-transaction": {
        "transactionShares": {
          "4": {
            "F2": "Shares withheld by BancorpSouth, Inc. to satisfy a tax withholding obligation resulting from the vesting of certain peformance shares.",
            "value": "<value>5595.0000</value><footnoteId id=\"F2\"/>"
          },
          "4A": {
            "value": "<value>8094.0000</value><footnoteId id=\"F1\"/>",
            "F1": "Shares withheld by BancorpSouth, Inc. to satisfy a tax withholding obligation resulting from the vesting of certain peformance shares."
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>178584.0000</value>"
          },
          "4A": {
            "value": "<value>176085.0000</value>"
          }
        }
      }
    },
    "accNum": "701853/0001225208-17-002148",
    "head": {},
    "dt": {}
  },
  "1295231/0001209191-17-007350": {
    "ndt": {
      "11-transaction": {
        "transactionPricePerShare": {
          "4": {
            "F23": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $813.33 to $814.32, inclusive.",
            "value": "<value>813.33</value><footnoteId id=\"F23\"/>"
          },
          "4A": {
            "value": "<value>813.33</value><footnoteId id=\"F11\"/>",
            "F11": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $813.33 to $814.32, inclusive."
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>20588823</value>"
          },
          "4A": {
            "value": "<value>20105487</value>"
          }
        }
      },
      "9-transaction": {
        "transactionPricePerShare": {
          "4": {
            "F21": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $809.05 to $810.04, inclusive.",
            "value": "<value>809.05</value><footnoteId id=\"F21\"/>"
          },
          "4A": {
            "F9": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $809.05 to $810.04, inclusive.",
            "value": "<value>809.05</value><footnoteId id=\"F9\"/>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>20589182</value>"
          },
          "4A": {
            "value": "<value>20105846</value>"
          }
        }
      },
      "7-transaction": {
        "transactionPricePerShare": {
          "4": {
            "value": "<value>806.7168</value><footnoteId id=\"F19\"/>",
            "F19": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $806.23 to $807.22, inclusive."
          },
          "4A": {
            "value": "<value>806.7168</value><footnoteId id=\"F7\"/>",
            "F7": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $806.23 to $807.22, inclusive."
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>20589582</value>"
          },
          "4A": {
            "value": "<value>20106246</value>"
          }
        }
      },
      "12-transaction": {
        "transactionPricePerShare": {
          "4": {
            "F24": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $814.65 to $815.64, inclusive.",
            "value": "<value>814.6598</value><footnoteId id=\"F24\"/>"
          },
          "4A": {
            "F12": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $814.65 to $815.64, inclusive.",
            "value": "<value>814.6598</value><footnoteId id=\"F12\"/>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>20588618</value>"
          },
          "4A": {
            "value": "<value>20105282</value>"
          }
        }
      },
      "2-transaction": {
        "transactionPricePerShare": {
          "4": {
            "F14": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $802.00 to $802.99, inclusive.",
            "value": "<value>802.4054</value><footnoteId id=\"F14\"/>"
          },
          "4A": {
            "F2": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $802.00 to $802.99, inclusive.",
            "value": "<value>802.4054</value><footnoteId id=\"F2\"/>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>20598347</value>"
          },
          "4A": {
            "value": "<value>20115011</value>"
          }
        }
      },
      "3-transaction": {
        "transactionPricePerShare": {
          "4": {
            "F15": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $799.96 to $800.95, inclusive.",
            "value": "<value>800.4521</value><footnoteId id=\"F15\"/>"
          },
          "4A": {
            "F3": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $799.96 to $800.95, inclusive.",
            "value": "<value>800.4521</value><footnoteId id=\"F3\"/>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>20597855</value>"
          },
          "4A": {
            "value": "<value>20114519</value>"
          }
        }
      },
      "10-transaction": {
        "transactionPricePerShare": {
          "4": {
            "value": "<value>811.18</value><footnoteId id=\"F22\"/>",
            "F22": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $810.95 to $811.94, inclusive."
          },
          "4A": {
            "F10": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $810.95 to $811.94, inclusive.",
            "value": "<value>811.18</value><footnoteId id=\"F10\"/>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>20588882</value>"
          },
          "4A": {
            "value": "<value>20105546</value>"
          }
        }
      },
      "1-transaction": {
        "transactionPricePerShare": {
          "4": {
            "F13": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $800.96 to $801.95, inclusive.",
            "value": "<value>801.4411</value><footnoteId id=\"F13\"/>"
          },
          "4A": {
            "value": "<value>801.4411</value><footnoteId id=\"F1\"/>",
            "F1": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $800.96 to $801.95, inclusive.  The Reporting Person undertakes to provide to any security holder of Alphabet Inc. or the staff of the Securities and Exchange Commission, upon request, full information regarding the number of shares sold at each separate price within the ranges set forth in footnotes (1) through (12) to this Form 4."
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>20602509</value>"
          },
          "4A": {
            "value": "<value>20119173</value>"
          }
        }
      },
      "6-transaction": {
        "transactionPricePerShare": {
          "4": {
            "F18": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $805.02 to $806.01, inclusive.",
            "value": "<value>805.4854</value><footnoteId id=\"F18\"/>"
          },
          "4A": {
            "value": "<value>805.4854</value><footnoteId id=\"F6\"/>",
            "F6": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $805.02 to $806.01, inclusive."
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>20591192</value>"
          },
          "4A": {
            "value": "<value>20107856</value>"
          }
        }
      },
      "4-transaction": {
        "transactionPricePerShare": {
          "4": {
            "value": "<value>803.4588</value><footnoteId id=\"F16\"/>",
            "F16": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $803.00 to $803.99, inclusive."
          },
          "4A": {
            "value": "<value>803.4588</value><footnoteId id=\"F4\"/>",
            "F4": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $803.00 to $803.99, inclusive."
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>20595913</value>"
          },
          "4A": {
            "value": "<value>20112577</value>"
          }
        }
      },
      "8-transaction": {
        "transactionPricePerShare": {
          "4": {
            "F20": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $807.27 to $808.26, inclusive.",
            "value": "<value>807.4667</value><footnoteId id=\"F20\"/>"
          },
          "4A": {
            "F8": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $807.27 to $808.26, inclusive.",
            "value": "<value>807.4667</value><footnoteId id=\"F8\"/>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>20589282</value>"
          },
          "4A": {
            "value": "<value>20105946</value>"
          }
        }
      },
      "5-transaction": {
        "transactionPricePerShare": {
          "4": {
            "value": "<value>804.5362</value><footnoteId id=\"F17\"/>",
            "F17": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $804.01 to $805.00, inclusive."
          },
          "4A": {
            "F5": "The price reported in Column 4 is a weighted average price. These shares were sold in multiple transactions at prices ranging from $804.01 to $805.00, inclusive.",
            "value": "<value>804.5362</value><footnoteId id=\"F5\"/>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>20593592</value>"
          },
          "4A": {
            "value": "<value>20110256</value>"
          }
        }
      }
    },
    "accNum": "1652044/0001209191-17-006770",
    "head": {},
    "dt": {}
  },
  "1684861/0001127602-17-005789": {
    "ndt": {
      "1-transaction": {
        "transactionShares": {
          "4": {
            "value": "<value>1956.7428</value>"
          },
          "4A": {
            "F2": "Amended to reflect actual shares withheld as a result of re-calculation of shares needed to satisfy federal taxes on distribution of performance shares.",
            "value": "<value>2104.1365</value><footnoteId id=\"F2\"/>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>26287.6705</value>"
          },
          "4A": {
            "value": "<value>26140.2768</value>"
          }
        }
      },
      "2-transaction": {
        "transactionShares": {
          "4": {
            "value": "<value>3665.2572</value>"
          },
          "4A": {
            "value": "<value>3517.8635</value><footnoteId id=\"F4\"/>",
            "F4": "Amended to reflect actual shares distributed in cash as a result of re-calculation of taxes."
          }
        }
      },
      "1-holding": {
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>5456.5433</value>"
          },
          "4A": {
            "value": "<value>5452.2777</value>"
          }
        }
      }
    },
    "accNum": "732717/0001127602-17-002975",
    "head": {},
    "dt": {}
  },
  "1659164/0001127602-17-004870": {
    "ndt": {
      "1-transaction": {
        "transactionShares": {
          "4": {
            "value": "<value>1025.99</value><footnoteId id=\"F1\"/>",
            "F1": "Represents shares of 3M common stock that will be delivered imminently pursuant to the terms of a performance share award made to the reporting person on March 3, 2014.  The performance-based vesting requirements applicable to such award were satisfied on the date reported in Column 2 above, which represents the date on which the level of performance attained was certified.  The number of shares withheld for taxes are estimated and will be revised by amendment, if necessary."
          },
          "4A": {
            "value": "<value>1025.99</value>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>5655</value>"
          },
          "4A": {
            "value": "<value>6681</value><footnoteId id=\"F1\"/>",
            "F1": "Due to a computational error caused by a third-party vendor's filing software, the reporting person is refiling a report filed earlier today, solely to correct total holdings reported in Column 5 of Table 1."
          }
        }
      },
      "2-transaction": {
        "transactionShares": {
          "4": {
            "value": "<value>495</value><footnoteId id=\"F1\"/>",
            "F1": "Represents shares of 3M common stock that will be delivered imminently pursuant to the terms of a performance share award made to the reporting person on March 3, 2014.  The performance-based vesting requirements applicable to such award were satisfied on the date reported in Column 2 above, which represents the date on which the level of performance attained was certified.  The number of shares withheld for taxes are estimated and will be revised by amendment, if necessary."
          },
          "4A": {
            "value": "<value>495</value>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "F2": "Includes shares acquired under 3M's General Employee Stock Purchase Plan.",
            "value": "<value>5655</value><footnoteId id=\"F2\"/>"
          },
          "4A": {
            "value": "<value>6186</value><footnoteId id=\"F1\"/>",
            "F1": "Due to a computational error caused by a third-party vendor's filing software, the reporting person is refiling a report filed earlier today, solely to correct total holdings reported in Column 5 of Table 1."
          }
        }
      },
      "1-holding": {
        "sharesOwnedFollowingTransaction": {
          "4": {
            "F3": "Includes shares acquired pursuant to the 3M Voluntary Investment Plan.",
            "value": "<value>254</value><footnoteId id=\"F3\"/>"
          },
          "4A": {
            "value": "<value>254</value>"
          }
        }
      }
    },
    "accNum": "66740/0001127602-17-004767",
    "head": {},
    "dt": {
      "1-transaction": {
        "securityTitle": {
          "4": {
            "value": "<value>Non-qualified Stock Option (Right to Buy)</value><footnoteId id=\"F4\"/>",
            "F4": "This option becomes exercisable in equal installments on each of the first three anniversaries of the grant date (2/7/2017)."
          },
          "4A": {
            "value": "<value>Non-qualified Stock Option (Right to Buy)</value>"
          }
        }
      }
    }
  },
  "1069533/0001201852-17-000004": {
    "ndt": {
      "1-transaction": {
        "sharesOwnedFollowingTransaction": {
          "4": {
            "F2": "Includes 94.643 restricted Shares purchased through dividends reinvested in the Restricted Stock Plan for Outside Directors of RGC Resources, Inc.",
            "value": "<value>21931.069</value><footnoteId id=\"F2\"/>"
          },
          "4A": {
            "F2": "Includes 94.643 restricted Shares purchased through dividends reinvested in the Restricted Stock Plan for Outside Directors of RGC Resources, Inc.",
            "F3": "Share total amended to include 1,000 shares of stock previously reported as UTMA on January 6, 2017 for the reporting person's children.",
            "value": "<value>22931.069</value><footnoteId id=\"F2\"/><footnoteId id=\"F3\"/>"
          }
        }
      }
    },
    "accNum": "1201852/0001201852-17-000002",
    "head": {
      "1": {
        "rptOwnerStreet2": {
          "4": {
            "value": "SUITE 210"
          },
          "4A": {
            "value": ""
          }
        },
        "rptOwnerCity": {
          "4": {
            "value": "HIGH POINT"
          },
          "4A": {
            "value": "ROANOKE"
          }
        },
        "rptOwnerZipCode": {
          "4": {
            "value": "27265"
          },
          "4A": {
            "value": "24030"
          }
        },
        "rptOwnerMailState": {
          "4": {
            "value": "NC"
          },
          "4A": {
            "value": "VA"
          }
        },
        "rptOwnerStreet1": {
          "4": {
            "value": "3980 PREMIER DRIVE"
          },
          "4A": {
            "value": "P.O. BOX 13007"
          }
        },
        "rptOwnerMailStreet2": {
          "4": {
            "value": "SUITE 210"
          },
          "4A": {}
        },
        "rptOwnerState": {
          "4": {
            "value": "NC"
          },
          "4A": {
            "value": "VA"
          }
        },
        "rptOwnerMailCity": {
          "4": {
            "value": "HIGH POINT"
          },
          "4A": {
            "value": "ROANOKE"
          }
        },
        "rptOwnerMailZip": {
          "4": {
            "value": "27265"
          },
          "4A": {
            "value": "24030"
          }
        },
        "rptOwnerMailStreet1": {
          "4": {
            "value": "3980 PREMIER DRIVE"
          },
          "4A": {
            "value": "P.O. BOX 13007"
          }
        }
      }
    },
    "dt": {}
  },
  "730272/0001140361-17-012884": {
    "ndt": {},
    "accNum": "1393547/0001140361-17-010119",
    "head": {},
    "dt": {
      "1-transaction": {
        "exerciseDate": {
          "4": {
            "F3": "The shares vest in equal annual installments over a three-year period beginning on the first anniversary of the grant date.",
            "value": "<footnoteId id=\"F3\"/>"
          },
          "4A": {
            "value": "<footnoteId id=\"F1\"/>",
            "F1": "The shares vest in equal annual installments over a three-year period beginning on the first anniversary of the grant date."
          }
        },
        "conversionOrExercisePrice": {
          "4": {
            "value": "<value>32.32</value>"
          },
          "4A": {
            "value": "<value>32.40</value>"
          }
        }
      }
    }
  },
  "1659164/0001127602-17-007824": {
    "ndt": {
      "1-transaction": {
        "transactionShares": {
          "4": {
            "value": "<value>1025.99</value><footnoteId id=\"F1\"/>",
            "F1": "Represents shares of 3M common stock that will be delivered imminently pursuant to the terms of a performance share award made to the reporting person on March 3, 2014.  The performance-based vesting requirements applicable to such award were satisfied on the date reported in Column 2 above, which represents the date on which the level of performance attained was certified.  The number of shares withheld for taxes are estimated and will be revised by amendment, if necessary."
          },
          "4A": {
            "value": "<value>1025.99</value>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>5655</value>"
          },
          "4A": {
            "value": "<value>6680.8475</value>"
          }
        }
      },
      "2-transaction": {
        "transactionShares": {
          "4": {
            "value": "<value>495</value><footnoteId id=\"F1\"/>",
            "F1": "Represents shares of 3M common stock that will be delivered imminently pursuant to the terms of a performance share award made to the reporting person on March 3, 2014.  The performance-based vesting requirements applicable to such award were satisfied on the date reported in Column 2 above, which represents the date on which the level of performance attained was certified.  The number of shares withheld for taxes are estimated and will be revised by amendment, if necessary."
          },
          "4A": {
            "value": "<value>521</value><footnoteId id=\"F1\"/>",
            "F1": "On February 9, 2017, the reporting person reported the imminent delivery of a number of shares of 3M Common Stock earned as a result of the 2014 performance shares awarded to the reporting person under the 3M Long-Term Incentive Plan.  The number of shares to be withheld from this delivery for the payment of withholding taxes was estimated at that time.  This amendment is being filed to update the Form 4 with the number of shares actually received after withholding taxes."
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "F2": "Includes shares acquired under 3M's General Employee Stock Purchase Plan.",
            "value": "<value>5655</value><footnoteId id=\"F2\"/>"
          },
          "4A": {
            "value": "<value>6159.8475</value>"
          }
        }
      },
      "1-holding": {
        "sharesOwnedFollowingTransaction": {
          "4": {
            "F3": "Includes shares acquired pursuant to the 3M Voluntary Investment Plan.",
            "value": "<value>254</value><footnoteId id=\"F3\"/>"
          },
          "4A": {
            "value": "<value>254</value>"
          }
        }
      }
    },
    "accNum": "66740/0001127602-17-004767",
    "head": {},
    "dt": {
      "1-transaction": {
        "securityTitle": {
          "4": {
            "value": "<value>Non-qualified Stock Option (Right to Buy)</value><footnoteId id=\"F4\"/>",
            "F4": "This option becomes exercisable in equal installments on each of the first three anniversaries of the grant date (2/7/2017)."
          },
          "4A": {
            "value": "<value>Non-qualified Stock Option (Right to Buy)</value>"
          }
        }
      }
    }
  },
  "1094988/0000947871-17-000047": {
    "ndt": {
      "1-transaction": {
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>4998751</value>"
          },
          "4A": {
            "value": "<value>4898751</value>"
          }
        }
      }
    },
    "accNum": "1438731/0000947871-17-000039",
    "head": {},
    "dt": {}
  },
  "1372455/0001127602-17-012935": {
    "ndt": {},
    "accNum": "1681689/0001127602-17-006628",
    "head": {},
    "dt": {
      "1-transaction": {
        "underlyingSecurityShares": {
          "4": {
            "value": "<value>26770</value>"
          },
          "4A": {
            "value": "<value>26818</value><footnoteId id=\"F1\"/>",
            "F1": "The number of derivative securities acquired, the amount or number of shares and the number of derivative securities beneficially owned following the reported transaction have been amended from the original filing."
          }
        },
        "transactionShares": {
          "4": {
            "value": "<value>26770</value>"
          },
          "4A": {
            "value": "<value>26818</value><footnoteId id=\"F1\"/>",
            "F1": "The number of derivative securities acquired, the amount or number of shares and the number of derivative securities beneficially owned following the reported transaction have been amended from the original filing."
          }
        },
        "exerciseDate": {
          "4": {
            "value": "<footnoteId id=\"F1\"/>",
            "F1": "This option vests in three equal annual installments beginning on February 14, 2018."
          },
          "4A": {
            "F2": "This option vests in three equal annual installments beginning on February 14, 2018.",
            "value": "<footnoteId id=\"F2\"/>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>26770</value>"
          },
          "4A": {
            "value": "<value>26818</value><footnoteId id=\"F1\"/>",
            "F1": "The number of derivative securities acquired, the amount or number of shares and the number of derivative securities beneficially owned following the reported transaction have been amended from the original filing."
          }
        }
      },
      "2-transaction": {
        "underlyingSecurityShares": {
          "4": {
            "value": "<value>2974</value>"
          },
          "4A": {
            "value": "<value>2980</value><footnoteId id=\"F1\"/>",
            "F1": "The number of derivative securities acquired, the amount or number of shares and the number of derivative securities beneficially owned following the reported transaction have been amended from the original filing."
          }
        },
        "transactionShares": {
          "4": {
            "value": "<value>2974</value>"
          },
          "4A": {
            "value": "<value>2980</value><footnoteId id=\"F1\"/>",
            "F1": "The number of derivative securities acquired, the amount or number of shares and the number of derivative securities beneficially owned following the reported transaction have been amended from the original filing."
          }
        },
        "exerciseDate": {
          "4": {
            "value": "<footnoteId id=\"F1\"/>",
            "F1": "This option vests in three equal annual installments beginning on February 14, 2018."
          },
          "4A": {
            "F2": "This option vests in three equal annual installments beginning on February 14, 2018.",
            "value": "<footnoteId id=\"F2\"/>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4": {
            "value": "<value>2974</value>"
          },
          "4A": {
            "value": "<value>2980</value><footnoteId id=\"F1\"/>",
            "F1": "The number of derivative securities acquired, the amount or number of shares and the number of derivative securities beneficially owned following the reported transaction have been amended from the original filing."
          }
        }
      }
    }
  }
}
import analyze
serverAna = analyze.runAnalysis(serverChanges)
print(json.dumps(serverAna, indent=2))

In [ ]:
print(analyze.getAllAnalyses(serverAna, fieldName="securityTitle")[0]["dt"])